In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
one_hot_data = pd.read_pickle("./one_hot.bz2")

In [ ]:
print one_hot_data.shape
one_hot_data = one_hot_data[one_hot_data.q9688_Nan != 1]
print one_hot_data.shape
one_hot_data.drop(['q9688_Nan'],axis=1, inplace=True )
one_hot_data

#https://stackoverflow.com/questions/38278965/selectively-zero-weights-in-tensorflow
#https://stackoverflow.com/questions/50001098/how-to-turn-off-individual-neurons-in-a-fully-connected-single-hidden-layer-n

In [20]:
from keras.models import Model
from keras.layers import Input, Dense, Lambda
from keras import layers
from keras.initializers import Ones, Zeros
import numpy as np

titles = list(one_hot_data)

# fix random seed for reproducibility
np.random.seed(7)
# create model

input_layer = Input(shape=(len(titles),))
hidden_layers = []

#a series of columns encoding a single question in one hot format
start_series = 0
question_columns = [] #Used to create the labels
while start_series < len(titles):
    if titles[start_series][0] != 'q':
#         print 'non q header', titles[start_series]
        start_series += 1
        continue    
    prefix = titles[start_series].split('_')[0]
    end_series = start_series
    while end_series < len(titles) and titles[end_series].startswith(prefix):
        end_series += 1

    mask = np.ones(len(titles))
    mask[start_series:end_series] = 0
    mask_lambda = Lambda(lambda x: x * mask)(input_layer)
    question_layer = Dense(end_series-start_series, activation='softmax')(mask_lambda)
#     question_layer = Dense(end_series-start_series, #TOPOLOGY TEST LAYER
#                            activation='linear',
#                            kernel_initializer=Ones(),
#                            bias_initializer=Zeros()
#                           )(mask_lambda)
    hidden_layers.append(question_layer) 
#     print prefix, question_layer, mask
    question_columns += list(range(start_series,end_series))
    
    start_series = end_series

if len(hidden_layers) > 1:
    concatenated_layers = layers.concatenate(hidden_layers)    
    model = Model(inputs=input_layer, outputs=concatenated_layers)
else:
    model = Model(inputs=input_layer, outputs=hidden_layers)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

#TOPOLOGY TEST, first 2 titles only
# a = np.array([[1,1,1,1,1,1]])
# print model.predict(a)
# a = np.array([[0,0,0,0,0,0]])
# print model.predict(a)
# a = np.array([[1,1,0,0,0,0]])
# print model.predict(a)
# a = np.array([[0,0,1,1,1,1]])
# print model.predict(a)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 428)               0         
_________________________________________________________________
lambda_10 (Lambda)           (None, 428)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 3)                 1287      
Total params: 1,287
Trainable params: 1,287
Non-trainable params: 0
_________________________________________________________________


In [19]:
# for i in zip(test.values, model.predict(np.array(test.values))):
#     print i
train, test = train_test_split(one_hot_data, test_size=0.05)
train_data = train.values
train_labels = train.iloc[:,question_columns].values
model.fit(train_data, train_labels, epochs=400, batch_size=1024*8, validation_split=0.05)

test_data = test.values
test_labels = test.iloc[:,question_columns].values


Train on 50266 samples, validate on 2646 samples
Epoch 1/400
50266/50266 [==============================] - 1s 17us/step - loss: 1.1810 - acc: 0.3594 - val_loss: 1.1245 - val_acc: 0.3783
Epoch 2/400
50266/50266 [==============================] - 1s 14us/step - loss: 1.0984 - acc: 0.3875 - val_loss: 1.0605 - val_acc: 0.4180
Epoch 3/400
50266/50266 [==============================] - 1s 14us/step - loss: 1.0434 - acc: 0.4313 - val_loss: 1.0214 - val_acc: 0.4535
Epoch 4/400
50266/50266 [==============================] - 1s 14us/step - loss: 1.0096 - acc: 0.4671 - val_loss: 0.9983 - val_acc: 0.4705
Epoch 5/400
50266/50266 [==============================] - 1s 14us/step - loss: 0.9888 - acc: 0.4848 - val_loss: 0.9836 - val_acc: 0.4815
Epoch 6/400
50266/50266 [==============================] - 1s 14us/step - loss: 0.9745 - acc: 0.4976 - val_loss: 0.9730 - val_acc: 0.4974
Epoch 7/400
50266/50266 [==============================] - 1s 14us/step - loss: 0.9637 - acc: 0.5109 - val_loss: 0.9643 - v

Epoch 118/400
50266/50266 [==============================] - 1s 13us/step - loss: 0.7627 - acc: 0.6372 - val_loss: 0.7764 - val_acc: 0.6130
Epoch 119/400
50266/50266 [==============================] - 1s 13us/step - loss: 0.7625 - acc: 0.6372 - val_loss: 0.7761 - val_acc: 0.6134
Epoch 120/400
50266/50266 [==============================] - 1s 14us/step - loss: 0.7622 - acc: 0.6371 - val_loss: 0.7759 - val_acc: 0.6130
Epoch 121/400
50266/50266 [==============================] - 1s 14us/step - loss: 0.7619 - acc: 0.6371 - val_loss: 0.7756 - val_acc: 0.6126
Epoch 122/400
50266/50266 [==============================] - 1s 14us/step - loss: 0.7616 - acc: 0.6371 - val_loss: 0.7754 - val_acc: 0.6138
Epoch 123/400
50266/50266 [==============================] - 1s 14us/step - loss: 0.7614 - acc: 0.6370 - val_loss: 0.7751 - val_acc: 0.6126
Epoch 124/400
50266/50266 [==============================] - 1s 14us/step - loss: 0.7611 - acc: 0.6374 - val_loss: 0.7749 - val_acc: 0.6138
Epoch 125/400
50266/

50266/50266 [==============================] - 1s 13us/step - loss: 0.7469 - acc: 0.6418 - val_loss: 0.7611 - val_acc: 0.6172
Epoch 235/400
50266/50266 [==============================] - 1s 14us/step - loss: 0.7468 - acc: 0.6421 - val_loss: 0.7610 - val_acc: 0.6183
Epoch 236/400
50266/50266 [==============================] - 1s 14us/step - loss: 0.7468 - acc: 0.6414 - val_loss: 0.7610 - val_acc: 0.6168
Epoch 237/400
50266/50266 [==============================] - 1s 14us/step - loss: 0.7468 - acc: 0.6414 - val_loss: 0.7609 - val_acc: 0.6164
Epoch 238/400
50266/50266 [==============================] - 1s 14us/step - loss: 0.7466 - acc: 0.6422 - val_loss: 0.7608 - val_acc: 0.6175
Epoch 239/400
50266/50266 [==============================] - 1s 14us/step - loss: 0.7466 - acc: 0.6422 - val_loss: 0.7609 - val_acc: 0.6156
Epoch 240/400
50266/50266 [==============================] - 1s 14us/step - loss: 0.7466 - acc: 0.6416 - val_loss: 0.7607 - val_acc: 0.6187
Epoch 241/400
50266/50266 [=======

50266/50266 [==============================] - 1s 14us/step - loss: 0.7432 - acc: 0.6420 - val_loss: 0.7583 - val_acc: 0.6175
Epoch 351/400
50266/50266 [==============================] - 1s 13us/step - loss: 0.7431 - acc: 0.6423 - val_loss: 0.7582 - val_acc: 0.6168
Epoch 352/400
50266/50266 [==============================] - 1s 14us/step - loss: 0.7431 - acc: 0.6423 - val_loss: 0.7584 - val_acc: 0.6164
Epoch 353/400
50266/50266 [==============================] - 1s 14us/step - loss: 0.7432 - acc: 0.6416 - val_loss: 0.7587 - val_acc: 0.6172
Epoch 354/400
50266/50266 [==============================] - 1s 14us/step - loss: 0.7431 - acc: 0.6420 - val_loss: 0.7586 - val_acc: 0.6172
Epoch 355/400
50266/50266 [==============================] - 1s 14us/step - loss: 0.7431 - acc: 0.6428 - val_loss: 0.7585 - val_acc: 0.6164
Epoch 356/400
50266/50266 [==============================] - 1s 14us/step - loss: 0.7431 - acc: 0.6426 - val_loss: 0.7585 - val_acc: 0.6172
Epoch 357/400
50266/50266 [=======

In [ ]:
# Creating the Confusion Matrix
y_pred = model.predict(test_data)
print 'test_labels', test_labels.shape
print 'y_pred', y_pred.shape


In [27]:
print titles[:len(y_pred[indx])]
for indx in range(0,20):
#     print np.array([int(round(x)) for x in y_pred[indx]])
    print y_pred[indx]
    print test_labels[indx]
    print

# from sklearn.metrics import confusion_matrix
# cm = confusion_matrix(test_labels, y_pred)

['q9688_Nan', 'q9688_No', 'q9688_Yes', 'q9688_Yes, but only soft stuff like marijuana', 'q12719_A LOT more than five', 'q12719_Fewer than five', 'q12719_More than five', 'q12719_Nan', 'q12719_None']
[0.01177816 0.06702553 0.14858666 0.7726096  0.05011497 0.22951235
 0.1378436  0.5573068  0.02522221]
[0 0 0 1 1 0 0 0 0]

[0.10827676 0.75153095 0.00954152 0.13065077 0.00233696 0.01977335
 0.00196152 0.9579418  0.01798637]
[0 1 0 0 0 0 0 1 0]

[0.04615132 0.34370404 0.03213957 0.578005   0.08482592 0.23165627
 0.08366365 0.554984   0.04487028]
[0 0 0 1 0 0 0 1 0]

[0.24949516 0.4308551  0.02292037 0.2967294  0.00296405 0.0342291
 0.00998823 0.93655753 0.01626108]
[0 0 0 1 0 0 0 1 0]

[2.3694235e-01 5.8165520e-01 2.4563950e-02 1.5683849e-01 9.2009775e-04
 1.0001450e-02 1.1040679e-03 9.7794616e-01 1.0028288e-02]
[0 1 0 0 0 0 0 1 0]

[0.1506693  0.3664006  0.03551441 0.44741562 0.01254326 0.1275259
 0.04479838 0.7716303  0.04350227]
[0 1 0 0 0 0 0 1 0]

[0.14007938 0.06354016 0.29051805 0.50